[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Wonder3D-colab/blob/main/Wonder3D_mesh_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Wonder3D
%cd /content/Wonder3D

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/random_states_0.pkl -d /content/Wonder3D/ckpts -o random_states_0.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/scaler.pt -d /content/Wonder3D/ckpts -o scaler.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/scheduler.bin -d /content/Wonder3D/ckpts -o scheduler.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/unet/diffusion_pytorch_model.bin -d /content/Wonder3D/ckpts/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/raw/main/unet/config.json -d /content/Wonder3D/ckpts/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/segment_anything/resolve/main/sam_vit_h_4b8939.pth -d /content/Wonder3D/sam_pt -o sam_vit_h_4b8939.pth

!pip install -q fire diffusers==0.19.3 transformers bitsandbytes accelerate gradio rembg segment_anything
!pip install -q einops omegaconf pytorch-lightning==1.9.5 torch_efficient_distloss nerfacc==0.3.3 PyMCubes trimesh
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/tinycudann-1.7-cp310-cp310-linux_x86_64.whl
!git clone https://huggingface.co/camenduru/torch_extensions_0_3_3_colab /root/.cache/torch_extensions

In [ ]:
from google.colab import files
from PIL import Image
import os

# Step 1: Upload the Image
print("Please upload a PNG file.")
%cd '/content/Wonder3D/example_images/'
uploaded = files.upload()

# Check if the uploaded file is a PNG
uploaded_filename = next(iter(uploaded))
if not uploaded_filename.lower().endswith('.png'):
    print("The uploaded file is not a PNG. Please upload a PNG file.")
else:
    # Display the uploaded image
    input_path = f'/content/Wonder3D/example_images/{uploaded_filename}'
    display(Image.open(input_path))

    # Step 2: Update Line 11 of the Configuration File
    config_file_path = '/content/Wonder3D/configs/mvdiffusion-joint-ortho-6views.yaml'

    with open(config_file_path, 'r') as file:
        lines = file.readlines()

    # Update line 11 with the new file name
    lines[10] = f"  filepaths: ['{uploaded_filename}']  # the test image names. leave it empty, test all images in the folder\n"

    with open(config_file_path, 'w') as file:
        file.writelines(lines)

    print("Configuration file updated successfully.")

In [ ]:
%cd /content/Wonder3D
!accelerate launch --config_file 1gpu.yaml test_mvdiffusion_seq.py --config /content/Wonder3D/configs/mvdiffusion-joint-ortho-6views.yaml

In [ ]:
%cd /content/Wonder3D/instant-nsr-pl
base_name = os.path.splitext(uploaded_filename)[0]
!python launch.py --config configs/neuralangelo-ortho-wmask.yaml --gpu 0 --train dataset.root_dir=/content/Wonder3D/outputs/cropsize-192-cfg3.0 dataset.scene={base_name}

In [ ]:
import os
import glob
from moviepy.editor import VideoFileClip, vfx
from google.colab import files
from IPython.display import Video

# Assuming 'uploaded_filename' is defined in a previous cell
base_name = os.path.splitext(uploaded_filename)[0]

# Base directories
normal_maps_dir = f'/content/Wonder3D/outputs/cropsize-192-cfg3.0/{base_name}/'
mesh_output_base_dir = f'/content/Wonder3D/instant-nsr-pl/exp/mesh-ortho-{base_name}/'

# Step 1: Download OBJ File
try:
    mesh_output_folders = next(os.walk(mesh_output_base_dir))[1]
    mesh_output_folder = os.path.join(mesh_output_base_dir, mesh_output_folders[0], 'save')
    obj_file = next(glob.iglob(f'{mesh_output_folder}/*.obj'), None)
    if obj_file:
        files.download(obj_file)
    else:
        print("No OBJ file found in the output folder.")
except Exception as e:
    print(f"Error while locating the OBJ file: {e}")

# Step 2: Display and Slow Down MP4 Video
try:
    video_file = next(glob.iglob(f'{mesh_output_folder}/*.mp4'), None)
    if video_file:
        clip = VideoFileClip(video_file)
        slowed_clip = clip.fx(vfx.speedx, 0.025)  # Slows down the video
        slowed_video_path = video_file.replace('.mp4', '-slowed.mp4')
        slowed_clip.write_videofile(slowed_video_path)

        print("\nDisplaying Slowed Down Video:")
        display(Video(slowed_video_path, embed=True))
    else:
        print("No MP4 video file found in the output folder.")
except Exception as e:
    print(f"Error while processing the video file: {e}")